In [ ]:
#Setting up the environment, dependencies and all prerequisites
!pip install pdfplumber
!pip install chromadb
!pip install tiktoken
!pip uninstall -y openai
!pip install --upgrade openai
#Reranking
!pip install sentence_transformers
# Import the CrossEncoder library from sentence_transformers
from sentence_transformers import CrossEncoder, util

import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import chromadb
import openai


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

from google.colab import userdata
api_key = userdata.get("OpenAI-Personal-Key")
client = openai.OpenAI(api_key = api_key)
openai.api_key = api_key


#Section for Functions Declarations Begins

# Define the function to generate the llm response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model
def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
                {"role": "user", "content": f"""You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{results_df}'. These search results are essentially a set of pages from an insurance pdf document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual page text from the policy document and the column 'metadatas' contains the following two key attributes :
                                                1) filing_name : Use this attribute as the Policy Name for forming the final citation in the final response
                                                2) Page_Number : Use this attribute to Add the relevant Page Numbers while forming the final citations in the final response

                                                Use the documents in '{results_df}' to answer the query '{query}'. Frame an informative answer and also use the dataframe to create the final citations including the following two
                                                1) Policy Name(s) : This comes from the filing_name attribute under the metadatas column within the dataframe and does not represent the page title
                                                2) Page Number(s) : This comes from the Page_Number attribute under the metadatas column within the dataframe. The value of Page Numbers will follow the following string format - Page XX - where XX represents the numeric page number

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular format.
                                                4. Use the Metadatas columns in the dataframe to retrieve and cite the policy name(s) and page numbers(s) as citation.
                                                5. The 'Policy Name' to tbe cited in the answer, will be picked from filing_name attribute found in the metadatas column within the dataframe
                                                6. The 'Page Number' to be cited in the answer, will be picked from Page_Number attribute found in the metadatas column within the dataframe
                                                7. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                8. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.
                                                9. The citation should appear only once in the final response. Do not duplicate the citation in the same response in any given form or format.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete summarized response first with all relevant information, and then finally provide the citations.
                                                """},
              ]

    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')


# Function to check whether a word is present in a table or not for segregation of regular text and tables
def check_bboxes(word, table_bbox):
  l = word['x0'], word['top'], word['x1'], word['bottom']
  r = table_bbox

  return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

# Function to extract text from a PDF file.
# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
  p = 0
  full_text = []

  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      page_no = f"Page {p+1}"
      text = page.extract_text().lower()#converting all text to lower for consistent search
      tables = page.find_tables()
      table_bboxes = [i.bbox for i in tables]

      tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
      non_table_words = [word for word in page.extract_words() if not any(
          [check_bboxes(word, table_bbox) for table_bbox in table_bboxes]
      )]
      lines = []

      for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance = 5):
        if 'text' in cluster[0]:
          try:
            lines.append(' '.join([i['text'] for i in cluster]))
          except KeyError:
            pass
        elif 'table' in cluster[0]:
          lines.append(json.dumps(cluster[0]['table']))
      full_text.append([page_no, ' '.join(lines)])

      p += 1
  return full_text

  import re

def extract_text_after_gc(texts, pageno):
    """
    Extracts substring starting after 'GC <number>' from each text in the list.
    If not found, returns the corresponding page number.

    Parameters:
    texts (list of str): List of input strings.
    pageno (list of str): List of corresponding page numbers.

    Returns:
    list of str: Extracted substrings or fallback page numbers.
    """
    extracted_texts = ""
    # Pattern: 'GC' + digits + capture rest of the line even across line breaks
    pattern = re.compile(r'GC\s*\d+\s+([^\n\r]+)', re.IGNORECASE)

    for text, page in zip(texts, pageno):
        match = pattern.search(text.replace('\n', ' '))  # Replace line breaks to keep it simple
        if match:
           # extracted_texts.append(match.group(1).strip())
           extracted_texts = match.group(1).strip()
        else:
            extracted_texts = str(pageno)
    return extracted_texts

def extract_text_after_update_phrase(texts, pageno):
    """
    Extracts substring starting after the phrase:
    'This policy has been updated effective January 1, 2014'
    from each text in the list. If not found, returns the corresponding page number.

    Parameters:
    texts (list of str): List of input strings.
    pageno (list of str): List of corresponding page numbers.

    Returns:
    list of str: Extracted substrings or fallback page numbers.
    """
    extracted_texts = ""

    # Regex to match the phrase and capture everything that follows
    pattern = re.compile(
        r'This\s+policy\s+has\s+been\s+updated\s+effective\s+January\s+1,\s*2014\s*(.*)',
        re.IGNORECASE | re.DOTALL
    )

    for text, page in zip(texts, pageno):
        match = pattern.search(text)
        if match:
            extracted_texts = match.group(1).strip()
        else:
            extracted_texts = str(pageno).lower()

    return extracted_texts

#Section for Functions Declarations Ends


#Loading of all PDF's data starts here

pdf_path = "/content/drive/MyDrive/GenAI_Course_Master/Course_2_HelpMateAICodes/Insurance Policy Documents"

# Define the directory containing the PDF files
pdf_directory = Path(pdf_path)

# Initialize an empty list to store the extracted texts and document names
data = []

# Loop through all files in the directory
for pdf_path in pdf_directory.glob("*.pdf"):

    # Process the PDF file
    print(f"...Processing {pdf_path.name}")

    # Call the function to extract the text from the PDF
    extracted_text = extract_text_from_pdf(pdf_path)

    # Convert the extracted list to a PD dataframe, and add a column to store document names
    extracted_text_df = pd.DataFrame(extracted_text, columns = ['Page Number', 'Page_Text'])
    extracted_text_df['Document Name'] = pdf_path.name

    # Append the extracted text and document name to the list
    data.append(extracted_text_df)

    # Print a message to indicate progress
    print(f"Finished Processing {pdf_path.name}")

# Print a message to indicate all PDFs have been processed
print("All PDFs have been processed")
#Loading of all PDF's data ends here


#Data Processing and Cleaning Starts Here
finance_pdfs_data = pd.concat(data, ignore_index = True)
finance_pdfs_data['Text_Length'] = finance_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))
finance_pdfs_data = finance_pdfs_data.loc[finance_pdfs_data['Text_Length'] >= 10]
finance_pdfs_data.head()
finance_pdfs_data.tail(5)
print(len(finance_pdfs_data))
#Data Processing and Cleaning Ends Here


#Importing of OpenAI embedding function starts here
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import re
chroma_data_path = "/content/drive/MyDrive/GenAI_Course_Master/Course_2_HelpMateAICodes"

# Call PersistentClient()
client = chromadb.PersistentClient(path = chroma_data_path)

# Set up the embedding function using the OpenAI embedding model
model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key = api_key, model_name=model)

## Create an empty collection
financedata_collection = client.get_or_create_collection(name = "RAG_on_insurance_documents", embedding_function = embedding_function)
print(financedata_collection)

# Store the metadata for each page in a separate column
finance_pdfs_data['Metadata'] = finance_pdfs_data.apply(lambda x: {'filing_name': x['Document Name'][:-4], 'Page_Number': x['Page Number'], 'Additional_Context_Information': extract_text_after_update_phrase([x['Page_Text']], [str(x['Page Number'])])}, axis=1)

documents_list = finance_pdfs_data['Page_Text'].tolist()
metadata_list = finance_pdfs_data['Metadata'].tolist()

financedata_collection.add(
    documents = documents_list,
    ids = [str(i) for i in range (0, len(documents_list))],
    metadatas = metadata_list
)
#Importing of OpenAI embedding function ends here
#Setting up the environment, dependencies and all prerequisites ends here

#Help Mate AI search Application Code starts here
#Run this cell to run the search application
print('Greetings! Let me know if you are looking for any information pertaining to Insurance policies and I will be happy to answer those questions for you!')
query = ''
user_input = ''
print('\n')
print('***********************************************************************************')
while len(query) == 0:
  query = input("Your Search Query : ")

while(query != "exit"):
      print('\n')
      print('HelpMateAI Answer : ')
      print('***********************************************************************************')
      results = financedata_collection.query(
            query_texts=query,
            include = ['embeddings', 'documents', 'metadatas','distances'],
            n_results=20
      )
      # Extract the top 20 results for the first query
      documents = results['documents'][0]
      metadatas = results['metadatas'][0]
      distances = results['distances'][0]

      # Initialise the cross encoder model
      cross_inputs = [[query, response] for response in documents]
      cross_rerank_scores = cross_encoder.predict(cross_inputs)
      cross_rerank_scores

      # Create a DataFrame from flattened results
      results_df = pd.DataFrame({
          'documents': documents,
          'metadatas': metadatas,
          'distances': distances,
          'Reranked_scores': cross_rerank_scores
      })
      # Return the top 3 results after reranking
      top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
      top_3_rerank[:3]

      top_3_RAG = top_3_rerank[["documents", "metadatas","distances","Reranked_scores"]][:3]

      response = generate_response(query, top_3_RAG)
      print("\n".join(response))
      print('***********************************************************************************')
      print('\n')
      query = input("Your Search Query : ")

      #Help Mate AI search Application Code ends here

      #Where can I find the death benefits extended to company members in the Principal-Sample-Life-Insurance-Policy document?
      #Where can i get the schedule of insurance for the Principal-Sample-Life-Insurance-Policy?
      #Where can i find information about Individual Purchase Rights in the Principal-Sample-Life-Insurance-Policy?


In [ ]:
top_3_RAG


documents  \
0  Section F - Individual Purchase Rights Article...   
4  I f a Dependent who was insured dies during th...   
2  M ember's death, the Death Benefits Payable ma...   

                                           metadatas  distances  \
0  {'Page_Number': 'Page 42', 'filing_name': 'Pri...   0.239691   
4  {'filing_name': 'Principal-Sample-Life-Insuran...   0.322693   
2  {'Page_Number': 'Page 47', 'Additional_Context...   0.305859   

   Reranked_scores  
0         2.990646  
4         1.791118  
2         0.730215